In [1]:
# =========================
# 1️⃣ Install Dependencies
# =========================
!pip install gspread oauth2client pydrive requests ffmpeg-python

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import subprocess, json, os, requests
from datetime import datetime

# =========================
# 2️⃣ Config
# =========================
SHEET_NAME = "VideoTopics"
SERVICE_ACCOUNT_FILE = "/content/service_account.json"  # Upload your service account JSON
VIDEO_OUTPUT_DIR = "/content/videos/"

if not os.path.exists(VIDEO_OUTPUT_DIR):
    os.makedirs(VIDEO_OUTPUT_DIR)

# =========================
# 3️⃣ Authenticate Google Sheets
# =========================
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, scope)
client = gspread.authorize(creds)
sheet = client.open(SHEET_NAME).sheet1

# =========================
# 4️⃣ Authenticate Google Drive
# =========================
gauth = GoogleAuth()
gauth.credentials = creds
drive = GoogleDrive(gauth)

# =========================
# 5️⃣ Fetch Rows from Sheet
# =========================
rows = sheet.get_all_records()
pending_rows = [r for r in rows if not r['Video_URL']]

# =========================
# 6️⃣ Helper Functions
# =========================
def download_file(url, filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    return filename

def merge_clips(clip_files, output_file):
    if len(clip_files) > 1:
        list_file = "/content/clip_list.txt"
        with open(list_file, 'w') as f:
            for cf in clip_files:
                f.write(f"file '{cf}'\n")
        subprocess.call(f"ffmpeg -y -f concat -safe 0 -i {list_file} -c copy {output_file}", shell=True)
    else:
        os.rename(clip_files[0], output_file)
    return output_file

def overlay_audio(video_file, tts_file, bgm_file, output_file):
    # Loop BGM to match video duration
    looped_bgm = f"/content/looped_bgm.mp3"
    subprocess.call(f"ffmpeg -y -stream_loop -1 -i {bgm_file} -t $(ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 {video_file}) {looped_bgm}", shell=True)
    # Merge TTS + BGM
    merged_audio = f"/content/merged_audio.mp3"
    subprocess.call(f"ffmpeg -y -i {tts_file} -i {looped_bgm} -filter_complex amix=inputs=2:duration=first:dropout_transition=2 {merged_audio}", shell=True)
    # Overlay on video
    subprocess.call(f"ffmpeg -y -i {video_file} -i {merged_audio} -c:v copy -map 0:v:0 -map 1:a:0 {output_file}", shell=True)
    return output_file

# =========================
# 7️⃣ Process Each Topic
# =========================
for idx, row in enumerate(pending_rows):
    topic = row['Topic']
    print(f"Processing topic: {topic}")

    try:
        # Load JSON fields
        narrations = json.loads(row['Narration'])
        clips_json = json.loads(row['Clips_URLs'])
        tts_json = json.loads(row['Audio_URL'])
        music_json = json.loads(row['Music_URL'])

        scene_videos = []

        for i, narration in enumerate(narrations):
            # Auto-select BGM if blank
            if not music_json[i]:
                if "war" in narration.lower(): music_json[i] = "https://www.pexels.com/audio/war.mp3"
                elif "divine" in narration.lower(): music_json[i] = "https://www.pexels.com/audio/divine.mp3"
                else: music_json[i] = "https://www.pexels.com/audio/default.mp3"

            # Download clips
            clip_files = [download_file(url, f"/content/clip_{i}_{j}.mp4") for j, url in enumerate(clips_json[i])]
            merged_clip = merge_clips(clip_files, f"/content/scene_{i}.mp4")

            # Download TTS & BGM
            tts_file = download_file(tts_json[i], f"/content/tts_{i}.mp3")
            bgm_file = download_file(music_json[i], f"/content/bgm_{i}.mp3")

            # Overlay audio
            final_scene = f"/content/final_scene_{i}.mp4"
            overlay_audio(merged_clip, tts_file, bgm_file, final_scene)
            scene_videos.append(final_scene)

        # Concatenate all scenes
        list_file = "/content/all_scenes.txt"
        with open(list_file, 'w') as f:
            for sv in scene_videos:
                f.write(f"file '{sv}'\n")
        final_video = f"{VIDEO_OUTPUT_DIR}{topic.replace(' ', '_')}.mp4"
        subprocess.call(f"ffmpeg -y -f concat -safe 0 -i {list_file} -c copy {final_video}", shell=True)

        # Upload to Google Drive
        gfile = drive.CreateFile({'title': os.path.basename(final_video)})
        gfile.SetContentFile(final_video)
        gfile.Upload()
        video_url = gfile['alternateLink']

        # Update Sheet
        sheet.update_cell(idx+2, 6, video_url)  # Video_URL
        sheet.update_cell(idx+2, 7, str(datetime.now()))  # Timestamp
        print(f"✅ Completed video for topic: {topic}")

    except Exception as e:
        print(f"❌ Error processing {topic}: {e}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=91204c3ac5cadfa35bb212d535370f694a9e65d00913293dc724efd17b26644e
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive
